In [8]:
import pandas as pd
import re
from tqdm import tqdm

def clean_catalog(text):
    if pd.isna(text):
        return ""

    # Normalize line breaks and spaces
    text = text.replace('\n', ' ').replace('\r', ' ').strip()
    
    # Split by common bullet or delimiter patterns
    parts = re.split(r'[•;|,•\-–●◆▶→\t]+', text)
    parts = [p.strip() for p in parts if len(p.strip()) > 1]

    # Try to make each specification human-readable
    clean_parts = []
    for p in parts:
        # If looks like key:value → reformat it
        if ":" in p:
            key, val = p.split(":", 1)
            clean_parts.append(f"{key.strip()} is {val.strip()}")
        elif "=" in p:
            key, val = p.split("=", 1)
            clean_parts.append(f"{key.strip()} is {val.strip()}")
        else:
            clean_parts.append(p)

    # Join into one line, separated by commas
    clean_text = ", ".join(clean_parts)

    # Remove extra spaces
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()

    return clean_text

# === MAIN ===
print("Loading data...")
df = pd.read_csv("dataset/test.csv")

print("Cleaning catalog content...")
tqdm.pandas()
df["catalog_content_clean"] = df["catalog_content"].progress_apply(clean_catalog)

# Keep only required columns
clean_df = df[["sample_id", "catalog_content_clean"]]

# Save cleaned CSV
output_path = "test/train_clean.csv"
clean_df.to_csv(output_path, index=False)

print(f"✅ Saved cleaned file as: {output_path}")
print(clean_df.head(5))


Loading data...
Cleaning catalog content...


100%|██████████| 75000/75000 [00:05<00:00, 14912.16it/s]


✅ Saved cleaned file as: test/train_clean.csv
   sample_id                              catalog_content_clean
0     100179  Item Name is Rani 14, Spice Eshamaya's Mango C...
1     245611  Item Name is Natural MILK TEA Flavoring extrac...
2     146263  Item Name is Honey Filled Hard Candy, Bulk Pac...
3      95658  Item Name is Vlasic Snack'mm's Kosher Dill 16 ...
4      36806  Item Name is McCormick Culinary Vanilla Extrac...


In [10]:
import pandas as pd
import re
from tqdm import tqdm

def clean_catalog(text):
    if pd.isna(text):
        return ""

    # Replace newlines and normalize spaces
    text = text.replace("\n", " ").replace("\r", " ").strip()

    # Remove field names like "Bullet Point 1:", "Bullet Point 2:", "Item Name:", "Value:", "Unit:" etc.
    text = re.sub(r'\b[Bb]ullet\s*[Pp]oint\s*\d*\s*:', '', text)
    text = re.sub(r'\b[Ii]tem\s*[Nn]ame\s*:', '', text)
    text = re.sub(r'\b[Vv]alue\s*:', '', text)
    text = re.sub(r'\b[Uu]nit\s*:', '', text)
    text = re.sub(r'\b[Bb]rand\s*:', '', text)
    text = re.sub(r'\b[Dd]escription\s*:', '', text)
    text = re.sub(r'\b[Ss]pecifications\s*:', '', text)

    # Split text on punctuation or bullet-like delimiters
    parts = re.split(r'[•;|,•\-–●◆▶→\t]+', text)
    parts = [p.strip() for p in parts if len(p.strip()) > 1]

    # Join into a single readable line
    clean_text = ", ".join(parts)

    # Remove redundant spaces and punctuation
    clean_text = re.sub(r'\s+', ' ', clean_text)
    clean_text = re.sub(r'\s+,', ',', clean_text)
    clean_text = clean_text.strip(' ,')

    return clean_text


# === MAIN ===
print("Loading data...")
df = pd.read_csv("dataset/test.csv")

print("Cleaning catalog_content column...")
tqdm.pandas()
df["catalog_content_clean"] = df["catalog_content"].progress_apply(clean_catalog)

# Keep only the desired columns
clean_df = df[["sample_id", "catalog_content_clean"]]

# Save the cleaned version
output_path = "test/test_clean.csv"
clean_df.to_csv(output_path, index=False)

print(f"✅ Cleaned dataset saved at: {output_path}")
print(clean_df.head(5))


Loading data...
Cleaning catalog_content column...


100%|██████████| 75000/75000 [00:14<00:00, 5170.28it/s]


✅ Cleaned dataset saved at: test/test_clean.csv
   sample_id                              catalog_content_clean
0     100179  Rani 14, Spice Eshamaya's Mango Chutney (India...
1     245611  Natural MILK TEA Flavoring extract by HALO PAN...
2     146263  Honey Filled Hard Candy, Bulk Pack 2 Pounds, I...
3      95658  Vlasic Snack'mm's Kosher Dill 16 Oz (Pack of 2...
4      36806  McCormick Culinary Vanilla Extract, 32 fl oz, ...


In [11]:
import pandas as pd
import re
from tqdm import tqdm

def clean_catalog(text):
    if pd.isna(text):
        return ""

    # Replace newlines and normalize spaces
    text = text.replace("\r", " ").replace("\n", " ").strip()

    # Remove field names and common prefixes
    text = re.sub(r'\b[Bb]ullet\s*[Pp]oint\s*\d*\s*:', '', text)
    text = re.sub(r'\b[Ii]tem\s*[Nn]ame\s*:', '', text)
    text = re.sub(r'\b[Vv]alue\s*:', '', text)
    text = re.sub(r'\b[Uu]nit\s*:', '', text)
    text = re.sub(r'\b[Bb]rand\s*:', '', text)
    text = re.sub(r'\b[Dd]escription\s*:', '', text)
    text = re.sub(r'\b[Ss]pecifications\s*:', '', text)

    # Split on patterns that indicate new items or bullet content
    parts = re.split(r'(?:[.?!]\s+|\s{2,}|,?\s*[•\-–●◆▶→]\s*|,?\s*Bullet\s*Point\s*\d*\s*)', text)
    parts = [p.strip() for p in parts if len(p.strip()) > 1]

    # Add comma after every bullet/segment
    clean_parts = []
    for p in parts:
        if not p.endswith(","):
            p = p + ","
        clean_parts.append(p)

    # Join them together
    clean_text = " ".join(clean_parts)

    # Clean redundant commas and spaces
    clean_text = re.sub(r'\s+,', ',', clean_text)
    clean_text = re.sub(r'\s+', ' ', clean_text)
    clean_text = clean_text.strip(' ,')

    return clean_text


# === MAIN ===
print("Loading training data...")
df = pd.read_csv("dataset/test.csv")

print("Cleaning catalog_content column...")
tqdm.pandas()
df["catalog_content_clean"] = df["catalog_content"].progress_apply(clean_catalog)

# Keep only desired columns
clean_df = df[["sample_id", "catalog_content_clean"]]

# Save to file
output_path = "test/test_clean2.csv"
clean_df.to_csv(output_path, index=False)

print(f"✅ Cleaned dataset saved as: {output_path}")
print(clean_df.head(3))


Loading training data...
Cleaning catalog_content column...


100%|██████████| 75000/75000 [00:20<00:00, 3688.72it/s]


✅ Cleaned dataset saved as: test/test_clean2.csv
   sample_id                              catalog_content_clean
0     100179  Rani 14, Spice Eshamaya's Mango Chutney (India...
1     245611  Natural MILK TEA Flavoring extract by HALO PAN...
2     146263  Honey Filled Hard Candy, Bulk Pack 2 Pounds, I...


In [12]:
import pandas as pd

# Load your cleaned dataset
df = pd.read_csv("test/test_clean2.csv")

def extract_product_name(text):
    if pd.isna(text) or not isinstance(text, str):
        return ""
    
    # Split by comma and take the first segment
    first_part = text.split(",")[0].strip()

    # Remove extra spaces or stray punctuation
    first_part = first_part.strip(' .,:;')
    return first_part

# Apply the extraction
df["product_name"] = df["catalog_content_clean"].apply(extract_product_name)

# Reorder columns (optional)
df = df[["sample_id", "product_name", "catalog_content_clean"]]

# Save to a new CSV
output_path = "test/test_with_product_name.csv"
df.to_csv(output_path, index=False)

print(f"✅ New dataset saved at: {output_path}")
print(df.head(5))


✅ New dataset saved at: test/test_with_product_name.csv
   sample_id                                       product_name  \
0     100179                                            Rani 14   
1     245611  Natural MILK TEA Flavoring extract by HALO PAN...   
2     146263                            Honey Filled Hard Candy   
3      95658    Vlasic Snack'mm's Kosher Dill 16 Oz (Pack of 2)   
4      36806                 McCormick Culinary Vanilla Extract   

                               catalog_content_clean  
0  Rani 14, Spice Eshamaya's Mango Chutney (India...  
1  Natural MILK TEA Flavoring extract by HALO PAN...  
2  Honey Filled Hard Candy, Bulk Pack 2 Pounds, I...  
3  Vlasic Snack'mm's Kosher Dill 16 Oz (Pack of 2...  
4  McCormick Culinary Vanilla Extract, 32 fl oz, ...  


In [13]:
import pandas as pd
import re

# Load the original train.csv
df = pd.read_csv("dataset/test.csv")

# Functions to extract "Value" and "Unit" fields
def extract_value(text):
    if pd.isna(text):
        return None
    match = re.search(r'Value\s*:\s*([0-9]*\.?[0-9]+)', text)
    if match:
        return match.group(1).strip()
    return None

def extract_unit(text):
    if pd.isna(text):
        return None
    match = re.search(r'Unit\s*:\s*([A-Za-z ]+)', text)
    if match:
        return match.group(1).strip()
    return None

# Apply the extraction
df["value"] = df["catalog_content"].apply(extract_value)
df["unit"] = df["catalog_content"].apply(extract_unit)

# Keep only required columns
value_unit_df = df[["value", "unit"]]

# Save to new CSV
output_path = "test/value_unit.csv"
value_unit_df.to_csv(output_path, index=False)

print(f"✅ New file saved at: {output_path}")
print(value_unit_df.head(10))


✅ New file saved at: test/value_unit.csv
   value   unit
0   10.5  Ounce
1    2.0  Fl Oz
2   32.0  Ounce
3    2.0  Count
4   32.0  Fl Oz
5   16.0  Ounce
6   45.0  Ounce
7    1.0  Count
8  180.0     oz
9   18.0  Ounce


In [14]:
import pandas as pd
import re

# Load original train.csv
df = pd.read_csv("dataset/test.csv")

# Functions to extract Value and Unit
def extract_value(text):
    if pd.isna(text):
        return 0
    match = re.search(r'Value\s*:\s*([0-9]*\.?[0-9]+)', text)
    if match:
        return float(match.group(1).strip())
    return 0  # default if not found

def extract_unit(text):
    if pd.isna(text):
        return "None"
    match = re.search(r'Unit\s*:\s*([A-Za-z ]+)', text)
    if match:
        return match.group(1).strip()
    return "None"  # default if not found

# Apply extraction
df["value"] = df["catalog_content"].apply(extract_value)
df["unit"] = df["catalog_content"].apply(extract_unit)

# Keep sample_id + value + unit
value_unit_df = df[["sample_id", "value", "unit"]]

# Save to CSV
output_path = "test/value_unit_with_id.csv"
value_unit_df.to_csv(output_path, index=False)

print(f"✅ New file saved at: {output_path}")
print(value_unit_df.head(10))


✅ New file saved at: test/value_unit_with_id.csv
   sample_id  value   unit
0     100179   10.5  Ounce
1     245611    2.0  Fl Oz
2     146263   32.0  Ounce
3      95658    2.0  Count
4      36806   32.0  Fl Oz
5     148239   16.0  Ounce
6      92659   45.0  Ounce
7       3780    1.0  Count
8     196940  180.0     oz
9      20472   18.0  Ounce


In [15]:
import pandas as pd

# Load the CSV
df = pd.read_csv("test/value_unit_with_id.csv")

# Convert all units to lowercase
df['unit'] = df['unit'].str.lower()

# Save back to CSV (overwrite or new file)
output_path = "test/value_unit_with_id_lower.csv"
df.to_csv(output_path, index=False)

print(f"✅ Units converted to lowercase and saved at: {output_path}")
print(df.head(10))


✅ Units converted to lowercase and saved at: test/value_unit_with_id_lower.csv
   sample_id  value   unit
0     100179   10.5  ounce
1     245611    2.0  fl oz
2     146263   32.0  ounce
3      95658    2.0  count
4      36806   32.0  fl oz
5     148239   16.0  ounce
6      92659   45.0  ounce
7       3780    1.0  count
8     196940  180.0     oz
9      20472   18.0  ounce


In [17]:
import pandas as pd

# Load the CSV
df = pd.read_csv("dataset/value_unit_standardized.csv")

# Total number of products
total_products = len(df)

# Count how many have 'None' as unit
none_count = (df['unit'] == "None").sum()
print(f"Total products: {total_products}")
print(f"Products with missing unit (None): {none_count}")
print(f"Percentage missing unit: {none_count / total_products * 100:.2f}%\n")

# Count of products per unit type
unit_counts = df['unit'].value_counts()
print("Count of products per unit type:")
print(unit_counts)

# Optional: percentage per unit
unit_percent = df['unit'].value_counts(normalize=True) * 100
print("\nPercentage of products per unit type:")
print(unit_percent)


Total products: 75000
Products with missing unit (None): 0
Percentage missing unit: 0.00%

Count of products per unit type:
unit
ounce                   44022
count                   18207
fl oz                   11332
none                      958
pound                     202
gm                         49
ct                         49
lb                         34
each                       26
pack                       21
ml                         15
bottle                      9
liters                      7
can                         6
kg                          6
bag                         6
jar                         4
pounds                      4
per carton                  3
box                         3
k                           3
case                        3
piece                       3
product                     2
sq ft                       2
pouch                       2
per box                     2
gr                          2
tea bags                    2
f

In [16]:
import pandas as pd

# Load CSV
df = pd.read_csv("test/value_unit_with_id_lower.csv")  # Already lowercase

# Mapping of variations → standardized units
unit_mapping = {
    "oz": "ounce",
    "ounces": "ounce",
    "fluid ounce": "fl oz",
    "fl": "fl oz",
    "fl oz": "fl oz",
    "fl ounce": "fl oz",
    "fluid ounces": "fl oz",
    "packs": "pack",
    "pack": "pack",
    "mililitro": "ml",
    "milliliter": "ml",
    "millilitre": "ml",
    "ml": "ml",
    "gram": "gm",
    "gramm": "gm",
    "grams": "gm",
    "gm": "gm",
    "none": "none"   # Keep 'none' as is
}

# Function to standardize units
def standardize_unit(unit):
    if pd.isna(unit):
        return "none"
    unit = unit.strip().lower()
    return unit_mapping.get(unit, unit)  # default to original if not in mapping

# Apply mapping
df['unit'] = df['unit'].apply(standardize_unit)

# Save cleaned CSV
output_path = "test/value_unit_standardized.csv"
df.to_csv(output_path, index=False)

print(f"✅ Standardized units saved at: {output_path}")
print(df['unit'].value_counts())


✅ Standardized units saved at: test/value_unit_standardized.csv
unit
ounce                                    43914
count                                    18187
fl oz                                    11406
none                                      1031
pound                                      200
ct                                          39
pack                                        36
gm                                          35
each                                        23
lb                                          18
ml                                          17
bottle                                       9
liters                                       7
bag                                          7
product                                      5
box                                          5
tea bags                                     4
foot                                         4
jar                                          4
ea                                    

In [18]:
import pandas as pd

# Load both CSVs
df_main = pd.read_csv("test/test_with_product_name.csv")  # sample_id, product_name, catalog_content_clean, price
df_value_unit = pd.read_csv("test/value_unit_standardized.csv")  # sample_id, value, unit

# Merge on sample_id (inner join keeps only matching rows)
merged_df = pd.merge(df_main, df_value_unit, on="sample_id", how="left")  # left join keeps all main rows

# Optional: reorder columns
merged_df = merged_df[["sample_id", "product_name", "catalog_content_clean", "value", "unit"]]

# Save merged CSV
output_path = "test/merged_train.csv"
merged_df.to_csv(output_path, index=False)

print(f"✅ Merged dataset saved at: {output_path}")
print(merged_df.head(5))


✅ Merged dataset saved at: test/merged_train.csv
   sample_id                                       product_name  \
0     100179                                            Rani 14   
1     245611  Natural MILK TEA Flavoring extract by HALO PAN...   
2     146263                            Honey Filled Hard Candy   
3      95658    Vlasic Snack'mm's Kosher Dill 16 Oz (Pack of 2)   
4      36806                 McCormick Culinary Vanilla Extract   

                               catalog_content_clean  value   unit  
0  Rani 14, Spice Eshamaya's Mango Chutney (India...   10.5  ounce  
1  Natural MILK TEA Flavoring extract by HALO PAN...    2.0  fl oz  
2  Honey Filled Hard Candy, Bulk Pack 2 Pounds, I...   32.0  ounce  
3  Vlasic Snack'mm's Kosher Dill 16 Oz (Pack of 2...    2.0  count  
4  McCormick Culinary Vanilla Extract, 32 fl oz, ...   32.0  fl oz  


In [19]:
import pandas as pd
import re

# Load your merged CSV
df = pd.read_csv("test/merged_test.csv")

# Function to remove emojis and special symbols
def remove_emojis(text):
    if pd.isna(text):
        return text
    # Remove all characters that are NOT basic printable ASCII
    return re.sub(r'[^\x00-\x7F]+', '', text)

# Apply to catalog_content_clean and product_name
df['catalog_content_clean'] = df['catalog_content_clean'].apply(remove_emojis)
df['product_name'] = df['product_name'].apply(remove_emojis)

# Save cleaned version
output_path = "test/merged_test_cleaned.csv"
df.to_csv(output_path, index=False)

print(f"✅ Cleaned dataset saved to: {output_path}")


✅ Cleaned dataset saved to: test/merged_test_cleaned.csv


In [20]:
import pandas as pd

# Load your merged CSV
df = pd.read_csv("test/merged_test_cleaned.csv")

# Add new column 'image' using sample_id
df['image'] = df['sample_id'].astype(str) + '.jpg'

# Save updated CSV
output_path = "test/merged_test_with_image.csv"
df.to_csv(output_path, index=False)

print(f"✅ Added image column using sample_id. Saved to: {output_path}")
print(df.head(5))


✅ Added image column using sample_id. Saved to: test/merged_test_with_image.csv
   sample_id                                       product_name  \
0     100179                                            Rani 14   
1     245611  Natural MILK TEA Flavoring extract by HALO PAN...   
2     146263                            Honey Filled Hard Candy   
3      95658    Vlasic Snack'mm's Kosher Dill 16 Oz (Pack of 2)   
4      36806                 McCormick Culinary Vanilla Extract   

                               catalog_content_clean  value   unit       image  
0  Rani 14, Spice Eshamaya's Mango Chutney (India...   10.5  ounce  100179.jpg  
1  Natural MILK TEA Flavoring extract by HALO PAN...    2.0  fl oz  245611.jpg  
2  Honey Filled Hard Candy, Bulk Pack 2 Pounds, I...   32.0  ounce  146263.jpg  
3  Vlasic Snack'mm's Kosher Dill 16 Oz (Pack of 2...    2.0  count   95658.jpg  
4  McCormick Culinary Vanilla Extract, 32 fl oz, ...   32.0  fl oz   36806.jpg  


In [21]:
import os
import pandas as pd

# Path to your image folder
IMAGE_FOLDER = "test_images/test_images"

# List all files in the folder
image_files = [f for f in os.listdir(IMAGE_FOLDER) if os.path.isfile(os.path.join(IMAGE_FOLDER, f))]

# Create a DataFrame
df_images = pd.DataFrame(image_files, columns=["image"])

# Save to CSV
output_path = "present_images.csv"
df_images.to_csv(output_path, index=False)

print(f"✅ Saved {len(df_images)} image names to {output_path}")


✅ Saved 74999 image names to present_images.csv


In [1]:
import pandas as pd

# Load the CSV files (adjust paths as needed)
first_csv_path = 'resnet_predictions.csv'  # Replace with your first CSV path
second_csv_path = 'p1.csv'  # Replace with your second CSV path (e.g., train.csv)

# Read CSVs
first_df = pd.read_csv(first_csv_path)
second_df = pd.read_csv(second_csv_path)

# Extract image names
first_images = set(first_df['image'].astype(str))  # Convert to set for efficient comparison
second_images = set(second_df['image'].astype(str))

# Find images in second CSV but not in first
missing_images = second_images - first_images

# Convert to list for output
missing_images_list = sorted(list(missing_images))

# Print results
print(f"Images in second CSV but not in first CSV ({len(missing_images_list)}):")
for img in missing_images_list:
    print(img)

# Optionally, save to a new CSV
output_df = pd.DataFrame({'image': missing_images_list})
output_df.to_csv('missing_images.csv', index=False)
print("Results saved to 'missing_images.csv'")

KeyError: 'image'

In [4]:
import pandas as pd

# Load your CSV file
df = pd.read_csv("predictions/sample_test_out.csv")

# Get data types of each column
print("Column Data Types:\n", df.dtypes)

# Separate integer and string columns
int_columns = df.select_dtypes(include=['int64', 'int32']).columns.tolist()
str_columns = df.select_dtypes(include=['object']).columns.tolist()

print("\n🧮 Integer Columns:", int_columns)
print("\n🔤 String Columns:", str_columns)


Column Data Types:
 sample_id      int64
price        float64
dtype: object

🧮 Integer Columns: ['sample_id']

🔤 String Columns: []


In [1]:
import torch
import torch.nn.functional as F
from transformers import AutoImageProcessor, AutoModel, AutoTokenizer
import os
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from PIL import Image, ImageFile
import numpy as np
from pathlib import Path

# --- Hugging Face warnings fixes ---
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
os.environ["HF_HUB_DISABLE_XET_WARNING"] = "1"

# Enable loading of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# --- Custom Collate Function ---
def custom_collate_fn(batch):
    images, texts, prices, sample_ids, image_paths = zip(*batch)
    return list(images), list(texts), torch.tensor(prices, dtype=torch.float32), list(sample_ids), list(image_paths)

# --- Product Dataset ---
class ProductDataset(Dataset):
    def __init__(self, df, image_folder, has_price=False):
        self.df = df
        self.image_folder = Path(image_folder)
        self.has_price = has_price  # Test dataset does not have price

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_name = str(row['image']).strip()  # remove stray spaces/tabs/newlines
        image_path = self.image_folder / image_name
        sample_id = row['sample_id']

        try:
            img = Image.open(image_path).convert('RGB')
        except Exception as e:
            print(f"❌ Error loading image {image_path} (sample_id: {sample_id}): {e}")
            img = Image.new('RGB', (224, 224), (0, 0, 0))  # fallback black image

        text = row.get('catalog_content_clean', "No description available")
        if pd.isna(text) or not isinstance(text, str) or text.strip() == "":
            text = "No description available"

        price = float(row['price']) if self.has_price else 0.0
        return img, text, price, sample_id, str(image_path)

# --- Embedding Extraction ---
def extract_and_save_embeddings(image_model, text_model, image_processor, text_tokenizer, data_loader, save_dir, max_length=128):
    image_model.eval()
    text_model.eval()
    os.makedirs(save_dir, exist_ok=True)
    
    # Detect last processed batch
    existing_batches = [
        int(f.split('_')[1])
        for f in os.listdir(save_dir)
        if f.startswith('batch_') and os.path.isdir(os.path.join(save_dir, f))
    ]
    start_batch = max(existing_batches) + 1 if existing_batches else 0
    print(f"🔁 Resuming from batch {start_batch}")

    metadata_path = os.path.join(save_dir, 'metadata.csv')
    processed_sample_ids = set()
    if os.path.exists(metadata_path):
        processed_metadata = pd.read_csv(metadata_path)
        processed_sample_ids = set(processed_metadata['sample_id'])
        print(f"Found {len(processed_sample_ids)} processed samples")

    all_image_embeddings = []
    all_text_embeddings = []
    all_sample_ids = []
    all_image_paths = []

    with torch.no_grad():
        for batch_idx, (batch_images, batch_texts, batch_prices, batch_sample_ids, batch_image_paths) in enumerate(
            tqdm(data_loader, desc="Extracting embeddings")
        ):
            if batch_idx < start_batch:
                continue
            if all(sample_id in processed_sample_ids for sample_id in batch_sample_ids):
                print(f"Skipping batch {batch_idx} (already processed)")
                continue

            print(f"Processing batch {batch_idx}")

            # Image embeddings
            try:
                inputs = image_processor(images=batch_images, return_tensors="pt").to(device)
                image_emb = image_model(**inputs).last_hidden_state.cpu().numpy()
            except Exception as e:
                print(f"❌ Error processing image embeddings for batch {batch_idx}: {e}")
                continue

            # Text embeddings
            try:
                inputs = text_tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to(device)
                text_emb = text_model(**inputs).last_hidden_state.cpu().numpy()
            except Exception as e:
                print(f"❌ Error processing text embeddings for batch {batch_idx}: {e}")
                continue

            # Save batch embeddings
            batch_save_dir = os.path.join(save_dir, f'batch_{batch_idx}')
            os.makedirs(batch_save_dir, exist_ok=True)
            torch.save(torch.tensor(image_emb), os.path.join(batch_save_dir, 'image_embeddings.pt'))
            torch.save(torch.tensor(text_emb), os.path.join(batch_save_dir, 'text_embeddings.pt'))

            all_image_embeddings.append(image_emb)
            all_text_embeddings.append(text_emb)
            all_sample_ids.extend(batch_sample_ids)
            all_image_paths.extend(batch_image_paths)

    # Save consolidated embeddings
    all_image_emb_path = os.path.join(save_dir, 'all_image_embeddings.pt')
    all_text_emb_path = os.path.join(save_dir, 'all_text_embeddings.pt')

    if all_image_embeddings:
        all_image_embeddings = np.concatenate(all_image_embeddings, axis=0)
        all_text_embeddings = np.concatenate(all_text_embeddings, axis=0)

        if os.path.exists(all_image_emb_path) and os.path.exists(all_text_emb_path):
            existing_image_emb = torch.load(all_image_emb_path).numpy()
            existing_text_emb = torch.load(all_text_emb_path).numpy()
            all_image_embeddings = np.concatenate([existing_image_emb, all_image_embeddings], axis=0)
            all_text_embeddings = np.concatenate([existing_text_emb, all_text_embeddings], axis=0)

        torch.save(torch.tensor(all_image_embeddings), all_image_emb_path)
        torch.save(torch.tensor(all_text_embeddings), all_text_emb_path)

        new_metadata = pd.DataFrame({
            'index': range(len(all_sample_ids)),
            'sample_id': all_sample_ids,
            'image_path': all_image_paths
        })

        if os.path.exists(metadata_path):
            existing_metadata = pd.read_csv(metadata_path)
            new_metadata = pd.concat([existing_metadata, new_metadata], ignore_index=True)
        new_metadata.to_csv(metadata_path, index=False)

    print(f"✅ Embeddings saved to {save_dir}")
    if all_image_embeddings:
        print(f"Image embeddings shape: {all_image_embeddings.shape} ([num_samples, 257, 768])")
        print(f"Text embeddings shape: {all_text_embeddings.shape} ([num_samples, {max_length}, 384])")
    else:
        print("No new embeddings generated (all batches skipped or empty)")

# --- Main Script ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load models
image_processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base', use_fast=True)
image_model = AutoModel.from_pretrained('facebook/dinov2-base').to(device)
print("DINOv2 model initialized.")

text_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')
text_model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2').to(device)
print("MiniLM model initialized.")

# Paths
EMBEDDINGS_DIR = './test_embeddings/'
os.makedirs(EMBEDDINGS_DIR, exist_ok=True)

IMAGE_FOLDER = 'test_images/test_images'  # Folder with all images
df = pd.read_csv("test/merged_test_with_image.csv")
print(f"Loaded {len(df)} rows")
print("Columns in CSV:", df.columns.tolist())

# Create dataset and dataloader
dataset = ProductDataset(df, IMAGE_FOLDER, has_price=False)
data_loader = DataLoader(dataset, batch_size=100, shuffle=False, num_workers=0, pin_memory=True, collate_fn=custom_collate_fn)
print("DataLoader ready.")

# Extract and save embeddings
extract_and_save_embeddings(
    image_model, text_model, image_processor, text_tokenizer,
    data_loader, EMBEDDINGS_DIR, max_length=128
)


'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /facebook/dinov2-base/resolve/main/preprocessor_config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001FBD0F7E3E0>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 5f693e30-5ff4-4e62-9054-2ace0f1ab1df)')' thrown while requesting HEAD https://huggingface.co/facebook/dinov2-base/resolve/main/preprocessor_config.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /facebook/dinov2-base/resolve/main/preprocessor_config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001FBD0ED1D50>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 603267a4-82bd-4ba9-9438-bc0a1ba0c52b)')' thrown while requesting HEAD https://huggingface.co/facebook/dinov2-b

DINOv2 model initialized.
MiniLM model initialized.
Loaded 74999 rows
Columns in CSV: ['sample_id', 'product_name', 'catalog_content_clean', 'value', 'unit', 'image']
DataLoader ready.
🔁 Resuming from batch 0


Extracting embeddings:   0%|          | 0/750 [00:00<?, ?it/s]

Processing batch 0


Extracting embeddings:   0%|          | 1/750 [00:12<2:30:08, 12.03s/it]

...

Extracting embeddings:  20%|██        | 152/750 [20:21<1:15:58,  7.62s/it]

Processing batch 152


Extracting embeddings:  20%|██        | 153/750 [20:29<1:16:28,  7.69s/it]

Processing batch 153


: 

In [1]:
import torch
import torch.nn.functional as F
from transformers import AutoImageProcessor, AutoModel, AutoTokenizer
import os
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from PIL import Image, ImageFile
import numpy as np
from pathlib import Path

# --- Hugging Face warnings fixes ---
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
os.environ["HF_HUB_DISABLE_XET_WARNING"] = "1"

# Enable loading of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# --- Custom Collate Function ---
def custom_collate_fn(batch):
    images, texts, prices, sample_ids, image_paths = zip(*batch)
    return list(images), list(texts), torch.tensor(prices, dtype=torch.float32), list(sample_ids), list(image_paths)

# --- Product Dataset ---
class ProductDataset(Dataset):
    def __init__(self, df, image_folder, has_price=False):
        self.df = df
        self.image_folder = Path(image_folder)
        self.has_price = has_price  # Test dataset does not have price

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_name = str(row['image']).strip()  # remove stray spaces/tabs/newlines
        image_path = self.image_folder / image_name
        sample_id = row['sample_id']

        try:
            img = Image.open(image_path).convert('RGB')
        except Exception as e:
            print(f"❌ Error loading image {image_path} (sample_id: {sample_id}): {e}")
            img = Image.new('RGB', (224, 224), (0, 0, 0))  # fallback black image

        text = row.get('catalog_content_clean', "No description available")
        if pd.isna(text) or not isinstance(text, str) or text.strip() == "":
            text = "No description available"

        price = float(row['price']) if self.has_price else 0.0
        return img, text, price, sample_id, str(image_path)

# --- Embedding Extraction ---
def extract_and_save_embeddings(image_model, text_model, image_processor, text_tokenizer, data_loader, save_dir, max_length=128):
    image_model.eval()
    text_model.eval()
    os.makedirs(save_dir, exist_ok=True)
    
    # Detect last processed batch
    existing_batches = [
        int(f.split('_')[1])
        for f in os.listdir(save_dir)
        if f.startswith('batch_') and os.path.isdir(os.path.join(save_dir, f))
    ]
    start_batch = max(existing_batches) + 1 if existing_batches else 0
    print(f"🔁 Resuming from batch {start_batch}")

    metadata_path = os.path.join(save_dir, 'metadata.csv')
    processed_sample_ids = set()
    if os.path.exists(metadata_path):
        processed_metadata = pd.read_csv(metadata_path)
        processed_sample_ids = set(processed_metadata['sample_id'])
        print(f"Found {len(processed_sample_ids)} processed samples")

    all_image_embeddings = []
    all_text_embeddings = []
    all_sample_ids = []
    all_image_paths = []

    with torch.no_grad():
        for batch_idx, (batch_images, batch_texts, batch_prices, batch_sample_ids, batch_image_paths) in enumerate(
            tqdm(data_loader, desc="Extracting embeddings")
        ):
            if batch_idx < start_batch:
                continue
            if all(sample_id in processed_sample_ids for sample_id in batch_sample_ids):
                print(f"Skipping batch {batch_idx} (already processed)")
                continue

            print(f"Processing batch {batch_idx}")

            # Image embeddings
            try:
                inputs = image_processor(images=batch_images, return_tensors="pt").to(device)
                image_emb = image_model(**inputs).last_hidden_state.cpu().numpy()
            except Exception as e:
                print(f"❌ Error processing image embeddings for batch {batch_idx}: {e}")
                continue

            # Text embeddings
            try:
                inputs = text_tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to(device)
                text_emb = text_model(**inputs).last_hidden_state.cpu().numpy()
            except Exception as e:
                print(f"❌ Error processing text embeddings for batch {batch_idx}: {e}")
                continue

            # Save batch embeddings
            batch_save_dir = os.path.join(save_dir, f'batch_{batch_idx}')
            os.makedirs(batch_save_dir, exist_ok=True)
            torch.save(torch.tensor(image_emb), os.path.join(batch_save_dir, 'image_embeddings.pt'))
            torch.save(torch.tensor(text_emb), os.path.join(batch_save_dir, 'text_embeddings.pt'))

            all_image_embeddings.append(image_emb)
            all_text_embeddings.append(text_emb)
            all_sample_ids.extend(batch_sample_ids)
            all_image_paths.extend(batch_image_paths)

    # Save consolidated embeddings
    all_image_emb_path = os.path.join(save_dir, 'all_image_embeddings.pt')
    all_text_emb_path = os.path.join(save_dir, 'all_text_embeddings.pt')

    if all_image_embeddings:
        all_image_embeddings = np.concatenate(all_image_embeddings, axis=0)
        all_text_embeddings = np.concatenate(all_text_embeddings, axis=0)

        if os.path.exists(all_image_emb_path) and os.path.exists(all_text_emb_path):
            existing_image_emb = torch.load(all_image_emb_path).numpy()
            existing_text_emb = torch.load(all_text_emb_path).numpy()
            all_image_embeddings = np.concatenate([existing_image_emb, all_image_embeddings], axis=0)
            all_text_embeddings = np.concatenate([existing_text_emb, all_text_embeddings], axis=0)

        torch.save(torch.tensor(all_image_embeddings), all_image_emb_path)
        torch.save(torch.tensor(all_text_embeddings), all_text_emb_path)

        new_metadata = pd.DataFrame({
            'index': range(len(all_sample_ids)),
            'sample_id': all_sample_ids,
            'image_path': all_image_paths
        })

        if os.path.exists(metadata_path):
            existing_metadata = pd.read_csv(metadata_path)
            new_metadata = pd.concat([existing_metadata, new_metadata], ignore_index=True)
        new_metadata.to_csv(metadata_path, index=False)

    print(f"✅ Embeddings saved to {save_dir}")
    if all_image_embeddings:
        print(f"Image embeddings shape: {all_image_embeddings.shape} ([num_samples, 257, 768])")
        print(f"Text embeddings shape: {all_text_embeddings.shape} ([num_samples, {max_length}, 384])")
    else:
        print("No new embeddings generated (all batches skipped or empty)")

# --- Main Script ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load models
image_processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base', use_fast=True)
image_model = AutoModel.from_pretrained('facebook/dinov2-base').to(device)
print("DINOv2 model initialized.")

text_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')
text_model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2').to(device)
print("MiniLM model initialized.")

# Paths
EMBEDDINGS_DIR = './test_embeddings/'
os.makedirs(EMBEDDINGS_DIR, exist_ok=True)

IMAGE_FOLDER = 'test_images/test_images'  # Folder with all images
df = pd.read_csv("test/merged_test_with_image.csv")
print(f"Loaded {len(df)} rows")
print("Columns in CSV:", df.columns.tolist())

# Create dataset and dataloader
dataset = ProductDataset(df, IMAGE_FOLDER, has_price=False)
data_loader = DataLoader(dataset, batch_size=100, shuffle=False, num_workers=0, pin_memory=True, collate_fn=custom_collate_fn)
print("DataLoader ready.")

# Extract and save embeddings
extract_and_save_embeddings(
    image_model, text_model, image_processor, text_tokenizer,
    data_loader, EMBEDDINGS_DIR, max_length=128
)


DINOv2 model initialized.
MiniLM model initialized.
Loaded 74999 rows
Columns in CSV: ['sample_id', 'product_name', 'catalog_content_clean', 'value', 'unit', 'image']
DataLoader ready.
🔁 Resuming from batch 153


Extracting embeddings:  20%|██        | 153/750 [05:09<19:06,  1.92s/it]

Processing batch 153


Extracting embeddings:  21%|██        | 154/750 [05:15<33:10,  3.34s/it]

Processing batch 154


...

Extracting embeddings:  40%|████      | 303/750 [23:41<55:48,  7.49s/it]

Processing batch 303


Extracting embeddings:  41%|████      | 304/750 [23:48<55:42,  7.49s/it]

Processing batch 304


: 

In [2]:
import torch
import torch.nn.functional as F
from transformers import AutoImageProcessor, AutoModel, AutoTokenizer
import os
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from PIL import Image, ImageFile
import numpy as np
from pathlib import Path

# --- Hugging Face warnings fixes ---
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
os.environ["HF_HUB_DISABLE_XET_WARNING"] = "1"

# Enable loading of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# --- Custom Collate Function ---
def custom_collate_fn(batch):
    images, texts, prices, sample_ids, image_paths = zip(*batch)
    return list(images), list(texts), torch.tensor(prices, dtype=torch.float32), list(sample_ids), list(image_paths)

# --- Product Dataset ---
class ProductDataset(Dataset):
    def __init__(self, df, image_folder, has_price=False):
        self.df = df
        self.image_folder = Path(image_folder)
        self.has_price = has_price  # Test dataset does not have price

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_name = str(row['image']).strip()  # remove stray spaces/tabs/newlines
        image_path = self.image_folder / image_name
        sample_id = row['sample_id']

        try:
            img = Image.open(image_path).convert('RGB')
        except Exception as e:
            print(f"❌ Error loading image {image_path} (sample_id: {sample_id}): {e}")
            img = Image.new('RGB', (224, 224), (0, 0, 0))  # fallback black image

        text = row.get('catalog_content_clean', "No description available")
        if pd.isna(text) or not isinstance(text, str) or text.strip() == "":
            text = "No description available"

        price = float(row['price']) if self.has_price else 0.0
        return img, text, price, sample_id, str(image_path)

# --- Embedding Extraction ---
def extract_and_save_embeddings(image_model, text_model, image_processor, text_tokenizer, data_loader, save_dir, max_length=128):
    image_model.eval()
    text_model.eval()
    os.makedirs(save_dir, exist_ok=True)
    
    # Detect last processed batch
    existing_batches = [
        int(f.split('_')[1])
        for f in os.listdir(save_dir)
        if f.startswith('batch_') and os.path.isdir(os.path.join(save_dir, f))
    ]
    start_batch = max(existing_batches) + 1 if existing_batches else 0
    print(f"🔁 Resuming from batch {start_batch}")

    metadata_path = os.path.join(save_dir, 'metadata.csv')
    processed_sample_ids = set()
    if os.path.exists(metadata_path):
        processed_metadata = pd.read_csv(metadata_path)
        processed_sample_ids = set(processed_metadata['sample_id'])
        print(f"Found {len(processed_sample_ids)} processed samples")

    all_image_embeddings = []
    all_text_embeddings = []
    all_sample_ids = []
    all_image_paths = []

    with torch.no_grad():
        for batch_idx, (batch_images, batch_texts, batch_prices, batch_sample_ids, batch_image_paths) in enumerate(
            tqdm(data_loader, desc="Extracting embeddings")
        ):
            if batch_idx < start_batch:
                continue
            if all(sample_id in processed_sample_ids for sample_id in batch_sample_ids):
                print(f"Skipping batch {batch_idx} (already processed)")
                continue

            print(f"Processing batch {batch_idx}")

            # Image embeddings
            try:
                inputs = image_processor(images=batch_images, return_tensors="pt").to(device)
                image_emb = image_model(**inputs).last_hidden_state.cpu().numpy()
            except Exception as e:
                print(f"❌ Error processing image embeddings for batch {batch_idx}: {e}")
                continue

            # Text embeddings
            try:
                inputs = text_tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to(device)
                text_emb = text_model(**inputs).last_hidden_state.cpu().numpy()
            except Exception as e:
                print(f"❌ Error processing text embeddings for batch {batch_idx}: {e}")
                continue

            # Save batch embeddings
            batch_save_dir = os.path.join(save_dir, f'batch_{batch_idx}')
            os.makedirs(batch_save_dir, exist_ok=True)
            torch.save(torch.tensor(image_emb), os.path.join(batch_save_dir, 'image_embeddings.pt'))
            torch.save(torch.tensor(text_emb), os.path.join(batch_save_dir, 'text_embeddings.pt'))

            all_image_embeddings.append(image_emb)
            all_text_embeddings.append(text_emb)
            all_sample_ids.extend(batch_sample_ids)
            all_image_paths.extend(batch_image_paths)

    # Save consolidated embeddings
    all_image_emb_path = os.path.join(save_dir, 'all_image_embeddings.pt')
    all_text_emb_path = os.path.join(save_dir, 'all_text_embeddings.pt')

    if all_image_embeddings:
        all_image_embeddings = np.concatenate(all_image_embeddings, axis=0)
        all_text_embeddings = np.concatenate(all_text_embeddings, axis=0)

        if os.path.exists(all_image_emb_path) and os.path.exists(all_text_emb_path):
            existing_image_emb = torch.load(all_image_emb_path).numpy()
            existing_text_emb = torch.load(all_text_emb_path).numpy()
            all_image_embeddings = np.concatenate([existing_image_emb, all_image_embeddings], axis=0)
            all_text_embeddings = np.concatenate([existing_text_emb, all_text_embeddings], axis=0)

        torch.save(torch.tensor(all_image_embeddings), all_image_emb_path)
        torch.save(torch.tensor(all_text_embeddings), all_text_emb_path)

        new_metadata = pd.DataFrame({
            'index': range(len(all_sample_ids)),
            'sample_id': all_sample_ids,
            'image_path': all_image_paths
        })

        if os.path.exists(metadata_path):
            existing_metadata = pd.read_csv(metadata_path)
            new_metadata = pd.concat([existing_metadata, new_metadata], ignore_index=True)
        new_metadata.to_csv(metadata_path, index=False)

    print(f"✅ Embeddings saved to {save_dir}")
    if all_image_embeddings:
        print(f"Image embeddings shape: {all_image_embeddings.shape} ([num_samples, 257, 768])")
        print(f"Text embeddings shape: {all_text_embeddings.shape} ([num_samples, {max_length}, 384])")
    else:
        print("No new embeddings generated (all batches skipped or empty)")

# --- Main Script ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load models
image_processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base', use_fast=True)
image_model = AutoModel.from_pretrained('facebook/dinov2-base').to(device)
print("DINOv2 model initialized.")

text_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')
text_model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2').to(device)
print("MiniLM model initialized.")

# Paths
EMBEDDINGS_DIR = './test_embeddings/'
os.makedirs(EMBEDDINGS_DIR, exist_ok=True)

IMAGE_FOLDER = 'test_images/test_images'  # Folder with all images
df = pd.read_csv("test/merged_test_with_image.csv")
print(f"Loaded {len(df)} rows")
print("Columns in CSV:", df.columns.tolist())

# Create dataset and dataloader
dataset = ProductDataset(df, IMAGE_FOLDER, has_price=False)
data_loader = DataLoader(dataset, batch_size=100, shuffle=False, num_workers=0, pin_memory=True, collate_fn=custom_collate_fn)
print("DataLoader ready.")

# Extract and save embeddings
extract_and_save_embeddings(
    image_model, text_model, image_processor, text_tokenizer,
    data_loader, EMBEDDINGS_DIR, max_length=128
)


DINOv2 model initialized.
MiniLM model initialized.
Loaded 74999 rows
Columns in CSV: ['sample_id', 'product_name', 'catalog_content_clean', 'value', 'unit', 'image']
DataLoader ready.
🔁 Resuming from batch 573


Extracting embeddings:  40%|████      | 302/750 [21:29<31:52,  4.27s/it]


KeyboardInterrupt: 